In [1]:
%reload_kedro --env intel

[05/18/24 18:07:12] INFO     Resolved project path as:                                              ]8;id=387764;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=568775;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#146\146]8;;\
                             /home/jamontanac/Documents/AdversarialExamples/adversarialexamples.                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/18/24 18:07:13] INFO     Kedro project AdversarialExamples                                      ]8;id=106001;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=417314;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#115\115]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=84194;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=490264;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#116\116]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=589139;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=713694;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#122\122]8;;\

In [2]:

import sys
sys.path.append('../')
import torch
import os
import itertools as it
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from notebooks.utils import randomFFT, compute_eigen_vector




def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

def plot_FFT_3d(avg_diff,std_diff,title = '', display = False,dataset = 'cifar',attack='DeepFool',model='Resnet'):
    x = np.arange(avg_diff.shape[1])
    y = np.arange(avg_diff.shape[0])
    # Create a figure with two subplots
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'surface'}, {'type': 'surface'}]],
        subplot_titles=('Average', 'Std'),
        horizontal_spacing=0.1,
        vertical_spacing=0.1
    )

    # Add first surface plot
    fig.add_trace(
        go.Surface(z=avg_diff, x=x, y=y, colorscale='Cividis',colorbar=dict(title='Z', x=0.42)),
        row=1, col=1
    )

    # Add second surface plot
    fig.add_trace(
        go.Surface(z=std_diff, x=x, y=y, colorscale='Cividis',colorbar=dict(title='Z2', x=1.02)),
        row=1, col=2
    )

    
    # Update layout, labels, and titles
    fig.update_layout(
        width=1200,  # Increase figure width
        height=600,  # Increase figure height
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            # camera=camera1
        ),
        scene2=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z2',
            # camera=camera2
        ),
        autosize=True
    )
    if display:
        fig.show()
    else:
        output_dir = f"Animations/FFT/{dataset}/{attack}/{model}/"
        ensure_dir(output_dir)
        safe_title = title.replace('/','_').replace("\\", "_")
        pio.write_image(fig, os.path.join(output_dir, f"{safe_title}.png"))



# def plot_eigen_vectors(avg_eigen_vectors,std_eigen_vectors,title = '',dataset="intel",display=False, cameraone= dict(x=2.25, y=1.25, z=0.25), cameratwo = dict(x=1.25, y=1.25, z=1.25)):
def plot_eigen_vectors(avg_eigen_vectors,std_eigen_vectors,title = '',dataset="intel",
                       display=False,attack='DeepFool',
                       model='Resnet'):

    avg_matrix = avg_eigen_vectors
    std_matrix = std_eigen_vectors
    log_avg_matrix = np.log1p(avg_matrix)
    log_std_matrix = np.log1p(std_matrix)
    x = np.arange(avg_matrix.shape[1])
    y = np.arange(avg_matrix.shape[0])
    # Create a figure with two subplots
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'surface'}, {'type': 'surface'}]],
        subplot_titles=('avg eigenvectors', 'std_eigen_vectors'),
        horizontal_spacing=0.1,
        vertical_spacing=0.1
    )

    # Add first surface plot
    fig.add_trace(
        go.Surface(z=log_avg_matrix, x=x, y=y, colorscale='Cividis',colorbar=dict(title='Log(Z)', x=0.42)),
        row=1, col=1
    )

    # Add second surface plot
    fig.add_trace(
        go.Surface(z=log_std_matrix, x=x, y=y, colorscale='Cividis',colorbar=dict(title='Log10(Z2)', x=1.02)),
        row=1, col=2
    )
    camera1 = dict(
        eye=dict(x=0.8, y=2., z=0.8),
        center=dict(x=0, y=0, z=0)
    )
    camera2 = dict(
        eye=dict(x=0.8, y=2., z=0.8),
        center=dict(x=0, y=0, z=0)
    )
    # Update layout, labels, and titles
    fig.update_layout(
        width=1200,  # Increase figure width
        height=600,  # Increase figure height
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Log(AVG)',
            camera=camera1
        ),
        scene2=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Log(STD)',
            camera=camera2
        ),
        autosize=True
    )
    if display:
        fig.show()
    else:
        output_dir = f"Animations/eigen/{dataset}/{attack}/{model}/"
        ensure_dir(output_dir)
        safe_title = title.replace('/','_').replace("\\", "_")
        pio.write_image(fig, os.path.join(output_dir, f"{safe_title}.png"))



# attacks = ["CarliniL2"]
attacks = ["DeepFool", "CarliniL2"]
models = ['Resnet','Regnet_x','Regnet_y']
image_size = catalog.load("params:Data_information.input_shape")
for model, attack in it.product(models, attacks):
    adversarial = catalog.load(f'{model}_model_Adversarial_{attack}@Dataset')
    original_images = adversarial['original']
    adversarial_images = adversarial['examples']
    average_difference = torch.zeros(*image_size)
    std_difference = torch.zeros(*image_size)
    average_eigen_vector = torch.zeros(*image_size)
    std_eigen_vectors = torch.zeros(*image_size)
    num_images = 0
    aux=0
    for i in range(len(original_images)):
        original_image = original_images[i]
        adversarial_image = adversarial_images[i]
        # noise_fft = randomFFT(torch.abs(adversarial_image-original_image),log=False)
        original_fft = randomFFT(original_image)
        adversarial_fft = randomFFT(adversarial_image)
        difference = torch.abs(adversarial_fft - original_fft)
        eig_image_noise = compute_eigen_vector(torch.abs(adversarial_image-original_image))
        average_difference += difference
        average_eigen_vector += torch.real(eig_image_noise)
        std_difference += difference**2
        std_eigen_vectors += torch.real(eig_image_noise)**2
        num_images += 1
        if num_images %10 ==0:

            aux+=1
            plot_FFT_3d(average_difference/num_images,
                        torch.sqrt(std_difference/num_images - (average_difference/num_images)**2),
                        title=f'{aux}'.zfill(3),
                        display=False,
                        dataset='intel',
                        attack=attack,
                        model=model)
            plot_eigen_vectors(average_eigen_vector/num_images,
                            torch.sqrt(std_eigen_vectors/num_images - (average_eigen_vector/num_images)**2),
                            title=f"{aux}".zfill(3),
                            display= False,
                            dataset='intel',
                            attack=attack,
                            model=model)
    
                            



                    INFO     Loading data from params:Data_information.input_shape              ]8;id=669036;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=72488;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from Resnet_model_Adversarial_DeepFool@Dataset        ]8;id=152275;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=27272;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 18:20:35] INFO     Loading data from Resnet_model_Adversarial_CarliniL2@Dataset       ]8;id=255719;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=139358;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 18:35:29] INFO     Loading data from Regnet_x_model_Adversarial_DeepFool@Dataset      ]8;id=124725;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=347437;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 18:50:44] INFO     Loading data from Regnet_x_model_Adversarial_CarliniL2@Dataset     ]8;id=233395;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=328756;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:04:10] INFO     Loading data from Regnet_y_model_Adversarial_DeepFool@Dataset      ]8;id=993559;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=447837;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:19:22] INFO     Loading data from Regnet_y_model_Adversarial_CarliniL2@Dataset     ]8;id=793269;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=927314;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

In [3]:

def plot_FFT(avg_diff,std_diff,size=(10,5),cmap = 'magma', title = '',
             file_name='',display = False,dataset = 'cifar',
             attack='DeepFool',model='Resnet'):
    fig, ax = plt.subplots(1, 2, figsize=size)
    # Plot the average difference
    cbar1 = ax[0].imshow(20*avg_diff, cmap=cmap, interpolation='nearest')
    fig.colorbar(cbar1, ax=ax[0],shrink=0.7)
    ax[0].axis('off')
    ax[0].set_title(f"Average")

    # Plot the std
    cbar2 = ax[1].imshow(20*std_diff, cmap=cmap, interpolation='nearest')
    # cbar2 = ax[1].imshow(instance, cmap=cmap, interpolation='nearest')
    fig.colorbar(cbar2, ax=ax[1],shrink=0.7)
    ax[1].axis('off')
    ax[1].set_title(f"Std")

    fig.suptitle(title)
    fig.tight_layout()
    if display:
        plt.show()
    else:
        output_dir = f"Animations/FFT_2d/{dataset}/{attack}/{model}/"
        ensure_dir(output_dir)
        safe_title = file_name.replace('/','_').replace("\\", "_")
        plt.savefig(os.path.join(output_dir, f"{safe_title}.png"))
        plt.close()
attacks = ["DeepFool", "CarliniL2", "FSGM", "PGD"]
# attacks = ["DeepFool"]
models = ['Resnet','Regnet_x','Regnet_y']
image_size = catalog.load("params:Data_information.input_shape")
for model, attack in it.product(models, attacks):
    adversarial = catalog.load(f'{model}_model_Adversarial_{attack}@Dataset')
    original_images = adversarial['original']
    adversarial_images = adversarial['examples']
    average_difference = torch.zeros(*image_size)
    std_difference = torch.zeros(*image_size)
    average_eigen_vector = torch.zeros(*image_size)
    std_eigen_vectors = torch.zeros(*image_size)
    num_images = 0
    aux=0
    for i in range(len(original_images)):
        original_image = original_images[i]
        adversarial_image = adversarial_images[i]
        # noise_fft = randomFFT(torch.abs(adversarial_image-original_image),log=False)
        original_fft = randomFFT(original_image)
        adversarial_fft = randomFFT(adversarial_image)
        difference = torch.abs(adversarial_fft - original_fft)
        eig_image_noise = compute_eigen_vector(torch.abs(adversarial_image-original_image))
        average_difference += difference
        average_eigen_vector += torch.real(eig_image_noise)
        std_difference += difference**2
        std_eigen_vectors += torch.real(eig_image_noise)**2
        num_images += 1
        if num_images %10 ==0:

            aux+=1
            plot_FFT(average_difference/num_images,
                        torch.sqrt(std_difference/num_images - (average_difference/num_images)**2),
                        title=f'FFT with N={num_images}',
                        display=False,
                        dataset='intel',
                        attack=attack,
                        model=model,
                        file_name=f"{aux}".zfill(3))

[05/18/24 19:33:57] INFO     Loading data from params:Data_information.input_shape              ]8;id=706786;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=430249;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from Resnet_model_Adversarial_DeepFool@Dataset        ]8;id=409989;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=905061;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:35:00] INFO     Loading data from Resnet_model_Adversarial_CarliniL2@Dataset       ]8;id=491975;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=337199;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:35:52] INFO     Loading data from Resnet_model_Adversarial_FSGM@Dataset            ]8;id=326012;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=646081;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:36:56] INFO     Loading data from Resnet_model_Adversarial_PGD@Dataset             ]8;id=335186;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=120765;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

...

[05/18/24 19:40:58] INFO     Loading data from Regnet_x_model_Adversarial_PGD@Dataset           ]8;id=195931;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=575830;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:41:57] INFO     Loading data from Regnet_y_model_Adversarial_DeepFool@Dataset      ]8;id=937827;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=339632;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:43:01] INFO     Loading data from Regnet_y_model_Adversarial_CarliniL2@Dataset     ]8;id=753623;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=683595;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:43:56] INFO     Loading data from Regnet_y_model_Adversarial_FSGM@Dataset          ]8;id=911152;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=746469;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/18/24 19:45:02] INFO     Loading data from Regnet_y_model_Adversarial_PGD@Dataset           ]8;id=810137;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=716981;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

In [5]:

%reload_kedro --env base

[05/16/24 02:27:38] INFO     Resolved project path as:                                              ]8;id=893554;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=810625;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#146\146]8;;\
                             /home/jamontanac/Documents/AdversarialExamples/adversarialexamples.                   
                             To set a different path, run '%reload_kedro <project_root>'                           

[05/16/24 02:27:40] INFO     Kedro project AdversarialExamples                                      ]8;id=305832;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=911233;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#115\115]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=376646;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=621824;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#116\116]8;;\
                             'pipelines'                                                                           

                    INFO     Registered line magic 'run_viz'                                        ]8;id=24028;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=442768;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/ipython/__init__.py#122\122]8;;\

In [6]:


attacks = ["DeepFool", "CarliniL2", "FSGM", "PGD"]
# attacks = ["DeepFool"]
models = ['Resnet','Regnet_x','Regnet_y']
image_size = catalog.load("params:Data_information.input_shape")
for model, attack in it.product(models, attacks):
    adversarial = catalog.load(f'{model}_model_Adversarial_{attack}@Dataset')
    original_images = adversarial['original']
    adversarial_images = adversarial['examples']
    average_difference = torch.zeros(*image_size)
    std_difference = torch.zeros(*image_size)
    average_eigen_vector = torch.zeros(*image_size)
    std_eigen_vectors = torch.zeros(*image_size)
    num_images = 0
    aux=0
    for i in range(len(original_images)):
        original_image = original_images[i]
        adversarial_image = adversarial_images[i]
        # noise_fft = randomFFT(torch.abs(adversarial_image-original_image),log=False)
        original_fft = randomFFT(original_image)
        adversarial_fft = randomFFT(adversarial_image)
        difference = torch.abs(adversarial_fft - original_fft)
        eig_image_noise = compute_eigen_vector(torch.abs(adversarial_image-original_image))
        average_difference += difference
        average_eigen_vector += torch.real(eig_image_noise)
        std_difference += difference**2
        std_eigen_vectors += torch.real(eig_image_noise)**2
        num_images += 1
        if num_images %10 ==0:

            aux+=1
            plot_FFT_3d(average_difference/num_images,
                        torch.sqrt(std_difference/num_images - (average_difference/num_images)**2),
                        title=f'{aux}'.zfill(3),
                        display=False,
                        dataset='cifar',
                        attack=attack,
                        model=model)
            plot_eigen_vectors(average_eigen_vector/num_images,
                            torch.sqrt(std_eigen_vectors/num_images - (average_eigen_vector/num_images)**2),
                            title=f"{aux}".zfill(3),
                            display= False,
                            dataset='cifar',
                            attack=attack,
                            model=model)

                    INFO     Loading data from params:Data_information.input_shape              ]8;id=210431;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=526662;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from Resnet_model_Adversarial_DeepFool@Dataset        ]8;id=344670;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=825866;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/16/24 03:07:38] INFO     Loading data from Resnet_model_Adversarial_CarliniL2@Dataset       ]8;id=757469;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=527506;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/16/24 03:50:58] INFO     Loading data from Resnet_model_Adversarial_FSGM@Dataset            ]8;id=139053;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=703584;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

[05/16/24 04:45:05] INFO     Loading data from Resnet_model_Adversarial_PGD@Dataset             ]8;id=144383;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=696933;file:///home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/kedro/io/data_catalog.py#482\482]8;;\
                             (PytorchDatasetDict)...                                                               

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:38                                                                                   │
│                                                                                                  │
│   35 │   │   │   │   │   │   dataset='cifar',                                                    │
│   36 │   │   │   │   │   │   attack=attack,                                                      │
│   37 │   │   │   │   │   │   model=model)                                                        │
│ ❱ 38 │   │   │   plot_eigen_vectors(average_eigen_vector/num_images,                             │
│   39 │   │   │   │   │   │   │   torch.sqrt(std_eigen_vectors/num_images - (average_eigen_vec    │
│   40 │   │   │   │   │   │   │   title=f"{aux}".zfill(3),                                        │
│   41 │   │   │   │   │   │   │   display= False,                                                 │
│                                                                                                  │
│ in plot_eigen_vectors:133                                                                        │
│                                                                                                  │
│   130 │   │   output_dir = f"Animations/eigen/{dataset}/{attack}/{model}/"                       │
│   131 │   │   ensure_dir(output_dir)                                                             │
│   132 │   │   safe_title = title.replace('/','_').replace("\\", "_")                             │
│ ❱ 133 │   │   pio.write_image(fig, os.path.join(output_dir, f"{safe_title}.png"))                │
│   134                                                                                            │
│   135                                                                                            │
│   136                                                                                            │
│                                                                                                  │
│ /home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/plotly/io/_kaleido.py: │
│ 266 in write_image                                                                               │
│                                                                                                  │
│   263 │   # Request image                                                                        │
│   264 │   # -------------                                                                        │
│   265 │   # Do this first so we don't create a file if image conversion fails                    │
│ ❱ 266 │   img_data = to_image(                                                                   │
│   267 │   │   fig,                                                                               │
│   268 │   │   format=format,                                                                     │
│   269 │   │   scale=scale,                                                                       │
│                                                                                                  │
│ /home/jamontanac/miniconda3/envs/AdvExamples/lib/python3.10/site-packages/plotly/io/_kaleido.py: │
│ 143 in to_image                                                                                  │
│                                                                                                  │
│   140 │   # Validate figure                                                                      │
│   141 │   # ---------------                                                                      │
│   142 │   fig_dict = validate_coerce_fig_to_dict(fig, validate)                                  │
│ ❱ 143 │   img_bytes = scope.transform(                                                           │
│   144 │   │   fig_dict, format=format, width=width, height=